# 从动作流（FlowAction）设计和实现的过程中思考函数组合的思想（草稿）

## 0. 关于动作流
- 前置动作，实现逻辑和后置动作，称为流(Flow)。
- 调用服务(InvokeService), 打开视图(OpenView), 关闭页面(Close), 跳转（Goto）称为动作（Action）
两者合起来称为 (FlowAction)

## 1. 动作流的Schema描述

### Before Flow

In [7]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

### Compute

In [8]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProducts',
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

### After

In [10]:
const afterFlow = {
    type: 'Flow',
    name: 'Effect',
    props: {},
    children: [
        {
            type: 'Action',
            name: 'OpenView',
            props: {
                viewKey: 'sayHello'
            }
        }
    ]
}

### Pipeline

In [11]:
const pipeline = {
    type: 'Flow',
    name: 'Pipeline',
    children: [
        beforeFlow,
        computeFlow,
        afterFlow
    ]
}

In [12]:
pipeline

{
  type: 'Flow',
  name: 'Pipeline',
  children: [
    {
      type: 'Flow',
      name: 'Validate',
      props: [Object],
      children: [Array]
    },
    {
      type: 'Flow',
      name: 'Compute',
      props: [Object],
      children: [Array]
    },
    { type: 'Flow', name: 'Effect', props: {}, children: [Array] }
  ]
}


### 类型定义

In [15]:
type ActionNode = {
    type: 'Action',
    name: string,
    props: Record<string, unknown>
}

type FlowNode = {
    type: 'Flow',
    name: string,
    props: Record<string, unknown>
    children: (FlowNode | ActionNode)[]
}

## 动作流的执行引擎

### 动作的定义

- 什么是动作？如何定义动作？
- 动作是一个操作，操作属于一种计算，函数即计算，所以动作可以用函数表达。—— 动作即函数！

### trival version

In [21]:
const validateData = (ctx, props) => {
    const record = ctx.record
    // 实际上会解析规则来执行
    return props.validate(record)
}



In [24]:
const invokeService = async (ctx, props) => {
    const route = ctx.route
    return props.invokeService(props.serviceKey)
}

In [28]:
const ctx = {
    record: { id: '1', name: 'iphone' },
    route: { module: 'products', action: 'show' }
}

## function composition

- Why?
- How?

### arity 1

- 什么是函数的类型？

In [32]:
const add1 = x => x + 1
const x2 = x => x * 2

In [35]:
// (x + 1) * 2

const f = x => x2(add1(x))
f(100)

202


In [34]:
const compose = (f, g) => x => f(g(x)) 

In [36]:
const f = compose(x2, add1)

In [37]:
f(100)

202


### curry

In [48]:
const add = (x, y, z) => x + y + z
const add2 = x => y => z => x + y + z

In [49]:
add(1,2,3)

6


In [50]:
add2(1)(2)(3)

6


### pointless

In [51]:
const range = n => [...Array(n).keys()]

In [52]:
range(10)

[
  0, 1, 2, 3, 4,
  5, 6, 7, 8, 9
]


In [53]:
const { faker } = require('@faker-js/faker')

[faker-js](https://fakerjs.dev/guide/usage.html) [faker-js/api](https://fakerjs.dev/api/)

In [54]:
faker.animal.cat()

Persian


In [65]:
const cats = range(100).map(_ => ({ id: faker.datatype.uuid(), name: faker.animal.cat(), age: faker.datatype.number(100), dark: faker.datatype.boolean() }))

In [93]:
cats.filter(x => x.age < 10).map(x => x.id)

[
  '8e53baad-c931-4e2b-a44e-2313e99c6654',
  'afa8ce05-3f41-4c88-b4d1-8b0186abb1db',
  '07be175d-b81c-43e9-a2e0-273cd3bcd796',
  '0a2bef0f-cc35-4c56-b3ef-e569bdd1d1d4',
  '8eb43676-3f1d-4089-b842-4d6da82f6b78',
  '9191352e-e583-4cdb-91e7-f73cff8e99d8',
  'cffc228c-583a-4ef5-b3b5-8fd1ed1c88c0',
  '1afb31e1-f958-4449-b85d-b12740a842d5',
  'ba7bff04-146f-4d34-9f09-75a440b5540e',
  '117269b5-2d15-452d-816f-67e80b61a759'
]


In [110]:
const findByAge = (list, age) => list.filter(x => x.age < age)

In [94]:
//findByAge(cats, 10)

In [111]:
const findByAge = list => age => list.filter(x => x.age < age)

//fiterWithAge(cats)(10)
findByAge(cats)

[Function (anonymous)]


In [112]:
const search = list => text => list.filter(x => x.name.includes(text))

In [98]:
//search(cats)('S')
search(cats)

[Function (anonymous)]


In [113]:
const fiindByAgeAndSearch = (list, age, text) => search(findByAge(list)(age))(text)

In [115]:
// fiindByAgeAndSearch(cats, 10, 'S')

In [116]:
const findByAge = age => list => list.filter(x => x.age < age)
const search = text => list => list.filter(x => x.name.includes(text))

In [119]:
const fiindByAgeAndSearch = (age, text) => compose(search(text), findByAge(age))

In [121]:
fiindByAgeAndSearch(10, 'S')(cats)

[
  {
    id: '8e53baad-c931-4e2b-a44e-2313e99c6654',
    name: 'Sphynx',
    age: 2,
    dark: true
  },
  {
    id: '8eb43676-3f1d-4089-b842-4d6da82f6b78',
    name: 'Exotic Shorthair',
    age: 2,
    dark: false
  },
  {
    id: '1afb31e1-f958-4449-b85d-b12740a842d5',
    name: 'Snowshoe',
    age: 2,
    dark: false
  }
]


### datalast

In [123]:
const filter = f => list => list.filter(f)
const map = f => list => list.map(f)
const take = n => list => list.slice(0, n)

In [127]:
const compose = (a, b, c) => x => a(b(c(x))) 

In [128]:
const get2DarkCatsWithS = compose(
    take(4),
    filter(x => x.name.includes('S')),
    filter(x => x.dark),
)

In [129]:
get2DarkCatsWithS(cats)

[
  {
    id: '8e53baad-c931-4e2b-a44e-2313e99c6654',
    name: 'Sphynx',
    age: 2,
    dark: true
  },
  {
    id: 'e638e320-2046-4e1d-92ce-86b6b484b830',
    name: 'Savannah',
    age: 85,
    dark: true
  },
  {
    id: '516121ab-c3fa-4700-b00d-8a89ee41d860',
    name: 'British Shorthair',
    age: 29,
    dark: true
  },
  {
    id: 'aadbc7da-83e7-4661-9ef6-f9888e635b26',
    name: 'British Shorthair',
    age: 43,
    dark: true
  }
]


## use ramda

In [142]:
const { map, filter, compose, prop, take, pick, sortBy } = require('ramda')

### transform data

In [140]:
const transform = map(pick(['id', 'name']))
// transform(cats)

In [160]:
const getData = compose(
    take(5),
    map(pick(['id', 'name'])),
    sortBy(prop('age')),
    filter(x => x.age > 10)
)

getData(cats)

[
  { id: '405c6691-fd40-45c3-a312-5af5c94f3939', name: 'Thai' },
  { id: '05236427-91df-41a1-addc-1839d76cb6e2', name: 'Burmese' },
  { id: 'c727e101-a862-4250-b3ed-bb6ca8d7eae1', name: 'Burmese' },
  { id: '112fe575-4c18-46c5-9e0e-35efd5f0efc0', name: 'Himalayan' },
  { id: '5fbc6332-1271-4a63-890c-67a29077dd31', name: 'Singapura' }
]


### curry

## 实现动作函数并组合